In [ ]:
# Set up environment

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [ ]:
# Define directory structure
base_dir = "/content/drive/MyDrive/11_LicensePlateDetection"
os.makedirs(base_dir, exist_ok=True)

In [ ]:
# Setup Folder Structure

import os
# Define directory structure
base_dir = "/content/drive/MyDrive/11_LicensePlateDetection"
os.makedirs(base_dir, exist_ok=True)

folders = ["frames", "annotations", "images", "labels", "yolo_results", "runs"]
for folder in folders:
    os.makedirs(os.path.join(base_dir, folder), exist_ok=True)

print("Folder structure created!")

In [ ]:
# Verify Filepaths (via Drive)

video_path = "/content/drive/MyDrive/11_LicensePlateDetection/DomDash.MOV"
output_folder = "/content/drive/MyDrive/11_LicensePlateDetection/frames"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Analyze the video

cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps
frame_duration = 1 / fps

# Print video properties
print(f"Frames Per Second (FPS): {fps}")
print(f"Total Frames: {total_frames}")
print(f"Video Duration: {video_duration:.2f} seconds")
print(f"Time per Frame: {frame_duration:.4f} seconds")

cap.release()  # Close video file

In [ ]:
# Extract frames

cap = cv2.VideoCapture(video_path)
frame_rate = 10  # Extract every 10th frame
frame_count = 0
extracted_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_rate == 0:
        frame_path = os.path.join(output_folder, f"frame_{extracted_count}.jpg")
        cv2.imwrite(frame_path, frame)
        extracted_count += 1

    frame_count += 1

cap.release()


print(f"Total Extracted Frames: {extracted_count}")

In [ ]:
# Get the resolution

if extracted_count > 0:
    first_frame_path = os.path.join(output_folder, "frame_0.jpg")
    first_frame = cv2.imread(first_frame_path)
    print(f"Video Resolution: {first_frame.shape[1]}x{first_frame.shape[0]} (Width x Height)")
else:
    print("Video Resolution: no frames extracted)")

In [ ]:
# Display the first 5 frames

sample_frames = []
for i in range(5):
    frame_path = os.path.join(output_folder, f"frame_{i*5}.jpg")
    img = cv2.imread(frame_path)
    sample_frames.append(img)

# Display first 5 extracted frames
fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for i, img in enumerate(sample_frames):
    axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[i].axis("off")
    axes[i].set_title(f"Frame {i*5}")
plt.show()

In [ ]:
# Prediction

# Paths
images_path = "/content/drive/MyDrive/11_LicensePlateDetection/frames"  # Extracted frames
results_path = "/content/drive/MyDrive/11_LicensePlateDetection/yolo_results"  # YOLO results
os.makedirs(results_path, exist_ok=True)

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Pre-trained YOLO model

# Run detection
for img in os.listdir(images_path):
    image_path = os.path.join(images_path, img)

    if img.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
      try:
        results = model.predict(image_path, save=True, conf=0.4, project=results_path)  # Save results in custom folder
      except Exception as e:
        print(f"Error processing {image_path}: {e}")
    else:
        print(f"Skipping non-image file: {img}")

print(f"YOLOv8 predictions saved in: {results_path}")

In [ ]:
# Label frames

# Define paths
images_folder = "/content/drive/MyDrive/11_LicensePlateDetection/frames"  # Path to extracted frames
labels_folder = "/content/drive/MyDrive/11_LicensePlateDetection/annotations"  # Folder to save YOLO labels
os.makedirs(labels_folder, exist_ok=True)

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

# Run detection on each frame
for img_file in os.listdir(images_folder):
    if img_file.lower().endswith((".jpg")):
        img_path = os.path.join(images_folder, img_file)
        try:
          results = model.predict(img_path, conf=0.3)

          # Get image dimensions
          img = cv2.imread(img_path)
          img_h, img_w, _ = img.shape  # Height, Width

          label_txt_path = os.path.join(labels_folder, img_file.replace(".jpg", ".txt").replace(".png", ".txt").replace(".jpeg",".txt"))

          # Open label file to write annotations
          with open(label_txt_path, "w") as label_file:
              for r in results:
                  for box in r.boxes.xyxy:
                      x1, y1, x2, y2 = box.tolist()

                      # Convert to YOLO format
                      x_center = ((x1 + x2) / 2) / img_w
                      y_center = ((y1 + y2) / 2) / img_h
                      width = (x2 - x1) / img_w
                      height = (y2 - y1) / img_h

                      # Write to label file
                      label_file.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
        except Exception as e:
          print(f"Error processing {img_path}: {e}")
    else:
      print(f"Skipping non-image file: {img_file}")
print("Auto-labeling complete! Check the 'labels/' folder for annotations.")

In [ ]:
# Split and Move Images and Labels

import shutil
import random

# Define dataset directories
frames_dir = os.path.join(base_dir, "frames")
annotations_dir = os.path.join(base_dir, "annotations")

# Define train, val, and test directories
train_images_dir = os.path.join(base_dir, 'images/train')
train_labels_dir = os.path.join(base_dir, 'labels/train')
val_images_dir = os.path.join(base_dir, 'images/val')
val_labels_dir = os.path.join(base_dir, 'labels/val')
test_images_dir = os.path.join(base_dir, 'images/test')
test_labels_dir = os.path.join(base_dir, 'labels/test')

# Create directories if they don’t exist
for folder in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir, test_images_dir, test_labels_dir]:
    os.makedirs(folder, exist_ok=True)

# Get all images from frames directory
image_files = [f for f in os.listdir(frames_dir) if f.endswith(".jpg")]
random.shuffle(image_files)  # Shuffle images for randomness

# Define dataset split sizes
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_size = int(len(image_files) * train_ratio)
val_size = int(len(image_files) * val_ratio)
test_size = len(image_files) - (train_size + val_size)

# Assign images to each dataset
train_files = image_files[:train_size]
val_files = image_files[train_size:train_size + val_size]
test_files = image_files[train_size + val_size:]

# Function to copy files instead of move
def copy_files(file_list, source_dir, dest_images_dir, dest_labels_dir):
    for file in file_list:
        img_path = os.path.join(source_dir, file)
        label_path = os.path.join(annotations_dir, file.replace(".jpg", ".txt"))

        # Copy image if exists
        if os.path.exists(img_path):
            shutil.copy(img_path, os.path.join(dest_images_dir, file))

        # Copy corresponding label if exists
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(dest_labels_dir, file.replace(".jpg", ".txt")))

# Copy images and labels into their respective folders
copy_files(train_files, frames_dir, train_images_dir, train_labels_dir)
copy_files(val_files, frames_dir, val_images_dir, val_labels_dir)
copy_files(test_files, frames_dir, test_images_dir, test_labels_dir)

# Verify the split
print(f"Dataset split complete!")
print(f"   Train: {len(train_files)} images")
print(f"   Validation: {len(val_files)} images")
print(f"   Test: {len(test_files)} images")

In [ ]:
# Train Yolo

# Load model
model = YOLO("yolov8n.pt")

# Train model and save outputs in Google Drive
model.train(data="/content/drive/MyDrive/11_LicensePlateDetection/data.yaml", epochs=25, imgsz=640, batch=16,
            project="/content/drive/MyDrive/11_LicensePlateDetection/runs", name="train")

In [ ]:
# Evaluate Model Performance

# Display training results
import matplotlib.pyplot as plt
import cv2
import glob

# Find the latest training run results
train_results_path = max(glob.glob('/content/drive/MyDrive/11_LicensePlateDetection/runs/train*'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()


In [ ]:
# Test Model on Test Images

# Load the trained model
model = YOLO('/content/drive/MyDrive/11_LicensePlateDetection/runs/train4/weights/best.pt')

# Run prediction
results = model.predict(
    source='/content/drive/MyDrive/11_LicensePlateDetection/test_image.jpg',  # image path
    conf=0.30,        # confidence threshold
    save=True,        # save result image
    save_txt=True,    # save prediction labels
    project='runs/detect',
    name='test_on_image',
    exist_ok=True
)

In [ ]:
from IPython.display import Image, display

# Show the image with bounding boxes
display(Image(filename='/content/runs/detect/test_on_image/test_image.jpg'))


## Roboflow Dataset


In [ ]:
# Set up environment

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [ ]:
# Define paths
import os
import shutil

project_name = "dataset"
drive_path = "/content/drive/MyDrive/0_Practicum2"
os.makedirs(drive_path, exist_ok=True)

# Download and unzip dataset directly to Drive
#!curl -L "x" > roboflow.zip
!unzip -q roboflow.zip -d "{os.path.join(drive_path, project_name)}" # Unzip directly to drive path
#!rm roboflow.zip # Remove the zip file

In [ ]:
# Train Yolo

# Load model
model = YOLO("yolov8n.pt")

# Train model and save outputs in Google Drive
model.train(data="/content/drive/MyDrive/0_Practicum2/dataset/data.yaml", epochs=25, imgsz=640, batch=16,
            project="/content/drive/MyDrive/0_Practicum2/dataset/runs", name="train")

# Evaluate Model Performance

In [ ]:
# Display training results
import matplotlib.pyplot as plt
import cv2
import glob

# Find the latest training run results
train_results_path = max(glob.glob('/content/drive/MyDrive/0_Practicum2/dataset/runs/train'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
# Confusion Matrix

performance_img = os.path.join(train_results_path, "confusion_matrix_normalized.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
# Continue training

# Load best model
model = YOLO("/content/drive/MyDrive/0_Practicum2/dataset/runs/train/weights/best.pt")

# Continue training
model.train(data="/content/drive/MyDrive/0_Practicum2/dataset/data.yaml",
            epochs=75,
            imgsz=640,
            batch=16,
            project="/content/drive/MyDrive/0_Practicum2/dataset/runs",
            name="more_training")

In [ ]:
# Display training results
import matplotlib.pyplot as plt
import cv2
import glob

# Find the latest training run results
train_results_path = max(glob.glob('/content/drive/MyDrive/0_Practicum2/dataset/runs/more_training'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
# Confusion Matrix

performance_img = os.path.join(train_results_path, "confusion_matrix_normalized.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
# Run validation
original_metrics = model_one.val(data="/content/drive/MyDrive/0_Practicum2/dataset/data.yaml")
continued_metrics = model_two.val(data="/content/drive/MyDrive/0_Practicum2/dataset/data.yaml")

# Access metrics via results_dict
orig = original_metrics.results_dict
cont = continued_metrics.results_dict


In [ ]:
# Compare models

print(f"{'Metric':<20} {'1st Model':<20} {'2nd Model'}")
print("-" * 60)
print(f"{'mAP@0.50':<20} {orig['metrics/mAP50(B)']:<20.4f} {cont['metrics/mAP50(B)']:.4f}")
print(f"{'mAP@0.50:0.95':<20} {orig['metrics/mAP50-95(B)']:<20.4f} {cont['metrics/mAP50-95(B)']:.4f}")
print(f"{'Precision':<20} {orig['metrics/precision(B)']:<20.4f} {cont['metrics/precision(B)']:.4f}")
print(f"{'Recall':<20} {orig['metrics/recall(B)']:<20.4f} {cont['metrics/recall(B)']:.4f}")

In [ ]:
# Box Plot

# Replace these with actual values from model.val().results_dict
original_metrics = {
    'mAP@0.50': 0.7131,
    'mAP@0.50:0.95': 0.3999,
    'Precision': 0.8977,
    'Recall': 0.6115
}

continued_metrics = {
    'mAP@0.50': 0.8100,
    'mAP@0.50:0.95': 0.4838,
    'Precision': 0.8546,
    'Recall': 0.7116
}

# Set up data
metrics_names = list(original_metrics.keys())
original_values = list(original_metrics.values())
continued_values = list(continued_metrics.values())

x = range(len(metrics_names))
bar_width = 0.35

# Plot
plt.figure(figsize=(10, 6))
plt.bar(x, original_values, width=bar_width, label='1st Model', color='blue')
plt.bar([i + bar_width for i in x], continued_values, width=bar_width, label='2nd Model', color='orange')

# Add labels, title, etc.
plt.xlabel('Metrics')
plt.ylabel('Scores')
plt.title('Model Performance')
plt.xticks([i + bar_width / 2 for i in x], metrics_names)
plt.ylim(0, 1.1)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


Next Steps:

*   Finish labeling images
*   Train model with larger dataset
*   Implement OCR




# Complete - Clean Dataset

In [ ]:
# Set up environment

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import glob

# Data Import in Yolov8 Format

In [ ]:
# Data Import (1691 images)

# Define paths
import os
import shutil

project_name = "customdata_yolov8"
drive_path = "/content/drive/MyDrive/0_Practicum2"
os.makedirs(drive_path, exist_ok=True)

# Download and unzip dataset directly to Drive
#!curl -L "xxx" > roboflow.zip
!unzip -q roboflow.zip -d "{os.path.join(drive_path, project_name)}" # Unzip directly to drive path
!rm roboflow.zip # Remove the zip file

## Yolov8n Training

In [ ]:
# Train Yolo

# Load model
model = YOLO("yolov8n.pt")

# Train model and save outputs in Google Drive
model.train(data="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/data.yaml", epochs=100, imgsz=640, batch=16,
            project="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs", name="v8n")

# Evaluate Yolov8n


In [ ]:
# Display training results
import matplotlib.pyplot as plt
import cv2
import glob

# Find the latest training run results
train_results_path = max(glob.glob('/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8n'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
print(f"mAP@0.5:        {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"Precision:      {metrics.box.mp:.4f}")
print(f"Recall:         {metrics.box.mr:.4f}")

In [ ]:
# Confusion Matrix

performance_img = os.path.join(train_results_path, "confusion_matrix_normalized.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

## Fine Tune

In [ ]:

model = YOLO("/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8n/weights/best.pt")

# Fine-tune settings
model.train(
    data="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    lr0=0.001,   # lower learning rate
    optimizer="SGD",
    augment=True,
    project="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs",
    name="v8n_finetuned",
    resume=False
)

# Evaluate Yolov8n - Refined

In [ ]:

train_results_path = max(glob.glob('/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8n_finetuned'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
print(f"mAP@0.5:        {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"Precision:      {metrics.box.mp:.4f}")
print(f"Recall:         {metrics.box.mr:.4f}")

In [ ]:
# Confusion Matrix

performance_img = os.path.join(train_results_path, "confusion_matrix_normalized.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

Recall is still under .60, mAP plateaued, classification is bouncing (taking on a lot noise)

# Yolov8m Training

In [ ]:
# Train Yolo

# Load model
model = YOLO("yolov8m.pt")

# Train model and save outputs in Google Drive
model.train(data="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/data.yaml", epochs=100, imgsz=640, batch=16,
            project="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs", name="v8m")

# Evaluate Yolov8m Performance

In [ ]:
train_results_path = max(glob.glob('/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8m'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
print(f"mAP@0.5:        {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"Precision:      {metrics.box.mp:.4f}")
print(f"Recall:         {metrics.box.mr:.4f}")

In [ ]:
metrics = model.val(data="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/data.yaml", verbose=True, plots=True)

In [ ]:
# Check number of val images
val_images = "/content/drive/MyDrive/0_Practicum2/customdata_yolov8/valid/images"
val_labels = "/content/drive/MyDrive/0_Practicum2/customdata_yolov8/valid/labels"

print("Number of val images:", len(os.listdir(val_images)))
print("Number of val labels:", len(os.listdir(val_labels)))


# Refine Yolov8m

In [ ]:
model = YOLO("/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8m/weights/best.pt")

# Continue training for 100 more epochs
model.train(
    data="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/data.yaml",
    epochs=100,
    patience=25,
    imgsz=640,
    batch=16,
    lr0=0.001,
    optimizer="SGD",
    augment=True,
    project="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs",
    name="v8m_refined",
    resume=False
)

In [ ]:
import glob

train_results_path = max(glob.glob('/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8m_refined'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
# Load the model
model = YOLO("/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8m_refined/weights/best.pt") # Assuming "best.pt" is your best model

# Run validation to get metrics
metrics = model.val(data="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/data.yaml")

print(f"mAP@0.5:        {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"Precision:      {metrics.box.mp:.4f}")
print(f"Recall:         {metrics.box.mr:.4f}")


# Model Comparison

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Metrics for all four model runs
data = {
    "Model": [
        "YOLOv8n_v1",
        "YOLOv8n_refined",
        "YOLOv8m_v1",
        "YOLOv8m_refined"
    ],
    "Precision": [0.81, 0.84, 0.87, 0.84],
    "Recall": [0.50, 0.56, 0.72, 0.59],
    "mAP@0.50": [0.65, 0.66, 0.88, 0.70],
    "mAP@0.50:0.95": [0.27, 0.33, 0.60, 0.36]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Bar positions and width
x = np.arange(len(df["Model"]))
width = 0.2

# Create the plot
fig, ax = plt.subplots(figsize=(12, 6))

# Plot each metric
ax.bar(x - 1.5*width, df["Precision"], width, label='Precision')
ax.bar(x - 0.5*width, df["Recall"], width, label='Recall')
ax.bar(x + 0.5*width, df["mAP@0.50"], width, label='mAP@0.50')
ax.bar(x + 1.5*width, df["mAP@0.50:0.95"], width, label='mAP@0.50:0.95')

# Labels, title, and legend
ax.set_ylabel('Score')
ax.set_title('YOLO Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(df["Model"], rotation=15)
ax.set_ylim(0, 1)
ax.legend(loc='upper left')
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


# Yolo 11 Testing

In [ ]:
import os
import shutil

project_name = "yolov11"
drive_path = "/content/drive/MyDrive/0_Practicum2"
os.makedirs(drive_path, exist_ok=True)

# Download and unzip dataset directly to Drive
#!curl -L "xxx" > roboflow.zip
!unzip -q roboflow.zip -d "{os.path.join(drive_path, project_name)}" # Unzip directly to drive path
!rm roboflow.zip # Remove the zip file

In [ ]:
import os
# Download the YOLOv11 weights file
!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov11.pt -P /content/drive/MyDrive/0_Practicum2/yolov11 # Download and place in your project directory

# Update the path to the YOLOv11 weights file in your code
model_path = "/content/drive/MyDrive/0_Practicum2/yolov11"
model = YOLO(model_path)

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!pip list | grep torch

In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

In [ ]:
# Set working directory
project_dir = "/content/drive/MyDrive/0_Practicum2/yolov11"
os.chdir(project_dir)

# Confirm current directory
print("Current directory:", os.getcwd())

In [ ]:
!yolo task=detect mode=train model=yolo11s.pt data="/content/drive/MyDrive/0_Practicum2/yolov11/data.yaml", epochs=50 imgsz=640 plots=True

In [ ]:
import glob

train_results_path = max(glob.glob('/content/drive/MyDrive/0_Practicum2/yolov11/runs/detect/train2'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
# Load the trained YOLO model
model = YOLO('/content/drive/MyDrive/0_Practicum2/yolov11/runs/detect/train2/weights/best.pt')  # Path to your best.pt file

# Run validation to get metrics
metrics = model.val(data="/content/drive/MyDrive/0_Practicum2/yolov11/data.yaml")  # Path to your data.yaml

# Now you can print the metrics
print(f"mAP@0.5:        {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"Precision:      {metrics.box.mp:.4f}")
print(f"Recall:         {metrics.box.mr:.4f}")

In [ ]:
model = YOLO('/content/drive/MyDrive/0_Practicum2/yolov11/runs/detect/train2/weights/best.pt')
model.train(
    data='/content/drive/MyDrive/0_Practicum2/yolov11/data.yaml',
    epochs=20,
    lr0=0.0005,
    patience=5,
    resume=False
)



In [ ]:
train_results_path = max(glob.glob('/content/drive/MyDrive/0_Practicum2/yolov11/runs/detect/train3'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Rollback to Previous Version

In [ ]:
train_results_path = max(glob.glob('/content/drive/MyDrive/0_Practicum2/yolov11/runs/detect/train2'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Yolov11m

In [ ]:
!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov11m.pt -P /content/drive/MyDrive/0_Practicum2/yolov11/

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11m.pt')

model.train(
    data='/content/drive/MyDrive/0_Practicum2/yolov11/data.yaml',
    epochs=100,
    imgsz=800,
    batch=16,
    patience=10,
    project='runs',
    name='yolov11m',
    lr0=0.001
)

In [ ]:
train_results_path = max(glob.glob('/content/drive/MyDrive/0_Practicum2/yolov11/runs/yolov11m'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
print(f"mAP@0.5:        {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"Precision:      {metrics.box.mp:.4f}")
print(f"Recall:         {metrics.box.mr:.4f}")

In [ ]:
metrics = model.val(
    data='data.yaml',
    conf=0.25,
    augment=True,
    iou=0.6
)
print(metrics.box)  # Precision, Recall, mAP50, mAP50-95


In [ ]:
print(f"mAP@0.5:        {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"Precision:      {metrics.box.mp:.4f}")
print(f"Recall:         {metrics.box.mr:.4f}")

# Compare all models

Run validation on each model

In [ ]:
from ultralytics import YOLO

# Define model names and paths
model_paths = {
    "YOLOv8n_v1": "/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8n/weights/best.pt",
    "YOLOv8n_refined": "/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8n_finetuned/weights/best.pt",
    "YOLOv8m_v1": "/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8m/weights/best.pt",
    "YOLOv8m_refined": "/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8m_refined/weights/best.pt",
    "YOLOv11s": "/content/drive/MyDrive/0_Practicum2/yolov11/runs/detect/train2/weights/best.pt",
    "YOLOv11m": "/content/drive/MyDrive/0_Practicum2/yolov11/runs/yolov11m/weights/best.pt"
}


results_dict = {}

for name, path in model_paths.items():
    print(f"Evaluating {name}...")
    model = YOLO(path)
    metrics = model.val(data='data.yaml', split='val', verbose=False)

    results_dict[name] = {
        'precision': metrics.box.mp,
        'recall': metrics.box.mr,
        'mAP@0.50': metrics.box.map50,
        'mAP@0.50:0.95': metrics.box.map
    }


Visualize the results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract metric names and model names
metrics = ['precision', 'recall', 'mAP@0.50', 'mAP@0.50:0.95']
model_names = list(results_dict.keys())

# Gather data
data = {metric: [results_dict[model][metric] for model in model_names] for metric in metrics}

# Plotting
x = np.arange(len(model_names))
width = 0.2

fig, ax = plt.subplots(figsize=(14, 6))

for i, metric in enumerate(metrics):
    ax.bar(x + i * width, data[metric], width, label=metric)

ax.set_ylabel('Score')
ax.set_title('YOLO Model Performance Comparison')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(model_names, rotation=15)
ax.set_ylim(0, 1.0)
ax.legend()
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()


Evaluate Models closer to training

In [ ]:
# Define model names and paths
model_paths = {
    "YOLOv8n_v1": "/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8n/weights/best.pt",
    "YOLOv8n_refined": "/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8n_finetuned/weights/best.pt",
    "YOLOv8m_v1": "/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8m/weights/best.pt",
    "YOLOv8m_refined": "/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8m_refined/weights/best.pt",
    "YOLOv11s": "/content/drive/MyDrive/0_Practicum2/yolov11/runs/detect/train2/weights/best.pt",
    "YOLOv11m": "/content/drive/MyDrive/0_Practicum2/yolov11/runs/yolov11m/weights/best.pt"
}


results_dict = {}

for name, path in model_paths.items():
    print(f"Evaluating {name}...")
    model = YOLO(path)
    metrics = model.val(data='data.yaml', split='val', conf=0.15, iou=0.6, augment=True, verbose=False)

    results_dict[name] = {
        'precision': metrics.box.mp,
        'recall': metrics.box.mr,
        'mAP@0.50': metrics.box.map50,
        'mAP@0.50:0.95': metrics.box.map
    }


In [ ]:
# Extract metric names and model names
metrics = ['precision', 'recall', 'mAP@0.50', 'mAP@0.50:0.95']
model_names = list(results_dict.keys())

# Gather data
data = {metric: [results_dict[model][metric] for model in model_names] for metric in metrics}

# Plotting
x = np.arange(len(model_names))
width = 0.2

fig, ax = plt.subplots(figsize=(14, 6))

for i, metric in enumerate(metrics):
    ax.bar(x + i * width, data[metric], width, label=metric)

ax.set_ylabel('Score')
ax.set_title('YOLO Model Performance Comparison')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(model_names, rotation=15)
ax.set_ylim(0, 1.0)
ax.legend()
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()

# Run Inference on Test Images

In [ ]:
model = YOLO('/content/drive/MyDrive/0_Practicum2/yolov11/runs/yolov11m/weights/best.pt')

# Inference on all images in a folder
results = model.predict(
    source='/content/drive/MyDrive/11_LicensePlateDetection/test_image.jpg',  # Folder with your test JPG/PNG files
    conf=0.25,                      # Confidence threshold (adjust if needed)
    save=True,                      # Saves results to /runs/detect/predict
    show=True                       # Displays results inline in notebook
)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

# Load the trained YOLOv11m model
model = YOLO('/content/drive/MyDrive/0_Practicum2/yolov11/runs/yolov11m/weights/best.pt')

# Run inference and save to custom folder
results = model.predict(
    source='/content/drive/MyDrive/11_LicensePlateDetection/test_image.jpg',
    conf=0.25,
    save=True,
    save_dir='/content/drive/MyDrive/0_Practicum2/Model_Images',
    show=True
)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

# Define the path to the saved YOLO result
predict_dir = '/content/runs/detect/predict5'

# List files in the directory
files = os.listdir(predict_dir)

# Find the first image (e.g., .jpg or .png)
image_file = next((f for f in files if f.lower().endswith(('.jpg', '.png'))), None)

# Load and display the image
if image_file:
    image_path = os.path.join(predict_dir, image_file)
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(12, 8))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title("YOLOv11m Detection Result")
    plt.show()
else:
    print("No image file found in:", predict_dir)


# Modifications

In [ ]:
from ultralytics import YOLO


model = YOLO("/content/drive/MyDrive/0_Practicum2/yolov11/yolo11m.pt")

model.train(
    data="/content/drive/MyDrive/0_Practicum2/yolov11/data.yaml",
    epochs=100,
    imgsz=800,
    batch=16,
    lr0=0.001,
    optimizer="SGD",
    augment=True,
    patience=20,
    project="runs",
    name="yolov11m_cleaned"
)


In [ ]:
train_results_path = max(glob.glob('/content/runs/yolov11m_cleaned'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
# Confusion Matrix

performance_img = os.path.join(train_results_path, "confusion_matrix_normalized.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Final Run

In [ ]:
import os
import shutil

project_name = "11_yolo"
drive_path = "/content/drive/MyDrive/0_Practicum2"
os.makedirs(drive_path, exist_ok=True)

# Download and unzip dataset directly to Drive
!curl -L "https://app.roboflow.com/ds/xhi0wmXd2s?key=G0Hg6KlAI8" > roboflow.zip
!unzip -q roboflow.zip -d "{os.path.join(drive_path, project_name)}" # Unzip directly to drive path
!rm roboflow.zip # Remove the zip file

Train Model

In [ ]:
model = YOLO("/content/drive/MyDrive/0_Practicum2/yolov11/yolo11m.pt")

model.train(
    data="/content/drive/MyDrive/0_Practicum2/11_yolo/data.yaml",
    epochs=150,
    imgsz=800,
    batch=16,
    lr0=0.001,
    optimizer="SGD",
    augment=True,
    patience=20,
    project="runs",
    name="11yolov_last"
)

# Evaluate

In [ ]:

model = YOLO("/content/runs/11yolov_last/weights/best.pt")

# Evaluate model
metrics = model.val(data="/content/drive/MyDrive/0_Practicum2/yolov11/data.yaml", split='val')


print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")


In [ ]:
train_results_path = max(glob.glob('/content/runs/11yolov_last'), key=os.path.getctime)

# Load training performance image
performance_img = os.path.join(train_results_path, "results.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
# Confusion Matrix

performance_img = os.path.join(train_results_path, "confusion_matrix_normalized.png")
img = cv2.imread(performance_img)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Validation Metrics

In [ ]:
from IPython.display import Image, display

display(Image(filename='/content/runs/detect/val/confusion_matrix_normalized.png'))

In [ ]:

import cv2
import matplotlib.pyplot as plt

# 1. Load your best trained model
model = YOLO("/content/drive/MyDrive/0_Practicum2/yolov11/runs/yolov11m_cleaned/weights/best.pt")

# 2. Set your test image path
test_image_path = "/content/drive/MyDrive/0_Practicum2/Nissan_Images/BasePic.jpg"

# 3. Run inference
results = model.predict(source=test_image_path, conf=0.05, save=False, show=False, verbose=False)[0]

# 4. Load and prepare the image
image = cv2.imread(test_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 5. Draw bounding boxes and confidence scores
for box, score in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
    x1, y1, x2, y2 = map(int, box[:4])
    confidence = score * 100  # convert to percentage
    label = f"{confidence:.1f}%"  # e.g., "97.5%"

    # Draw rectangle
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Put confidence text
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

# 6. Show the final image
plt.figure(figsize=(12, 8))
plt.imshow(image)
plt.axis("off")
plt.title("YOLOv11m Detection with Confidence")
plt.show()


# April Run

In [ ]:
# Define paths
import os
import shutil

project_name = "April_Run"
drive_path = "/content/drive/MyDrive/0_Practicum2"
os.makedirs(drive_path, exist_ok=True)

# Download and unzip dataset directly to Drive
#!curl -L "xxx" > roboflow.zip
!unzip -q roboflow.zip -d "{os.path.join(drive_path, project_name)}" # Unzip directly to drive path
!rm roboflow.zip # Remove the zip file


In [ ]:
# Train Yolo

# Load model
model = YOLO("yolov8m.pt")

# Train model and save outputs in Google Drive
model.train(data="/content/drive/MyDrive/0_Practicum2/dataset/data.yaml", epochs=150, imgsz=640, batch=16, patience=20,
            project="/content/drive/MyDrive/0_Practicum2/April_Run/runs", name="train")

In [ ]:
results = model.val(data="/content/drive/MyDrive/0_Practicum2/April_Run/data.yaml", split='val')

In [ ]:
# Load the model
model = YOLO("/content/drive/MyDrive/0_Practicum2/yolov11/runs/yolov11m_cleaned/weights/best.pt")

# Load and convert the image
image_path = '/content/drive/MyDrive/0_Practicum2/Nissan_Images/BasePic.jpg'
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run detection
results = model(image_path, conf=0.01, verbose=False)[0]

# Check if any boxes were detected
if results.boxes is None or len(results.boxes.xyxy) == 0:
    print("No license plates detected.")
else:
    print(f"{len(results.boxes.xyxy)} object(s) detected.")

    # Draw bounding boxes
    for box in results.boxes.xyxy.cpu().numpy():
        x1, y1, x2, y2 = map(int, box[:4])
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Show image with bounding boxes
    plt.figure(figsize=(12, 8))
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.title("YOLOv11 Detection Test")
    plt.show()


In [ ]:
val_results = model.val(data='/content/drive/MyDrive/0_Practicum2/yolov11/data.yaml')
print(f"mAP@0.5: {val_results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {val_results.box.map:.4f}")


In [ ]:
# Load a pretrained general YOLOv8 model
pretrained_model = YOLO('yolov8m.pt')  # or yolov8m.pt

# Predict on your image
pretrained_model.predict(source='/content/drive/MyDrive/0_Practicum2/Nissan_Images/BasePic.jpg', conf=0.25, show=True)


In [ ]:

model = YOLO('yolov8m.pt')

model.train(
    data="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/data.yaml",
    epochs=100,
    imgsz=800,
    batch=16,
    lr0=0.001,
    optimizer="SGD",
    augment=True,
    patience=25,
    project="/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs",
    name="v8_Retrain_April",
    verbose=True
)


# Validate

In [ ]:
!pip install ultralytics

from ultralytics import YOLO

In [ ]:
model = YOLO('/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_Retrain_April/weights/best.pt')
results = model.val(data='/content/drive/MyDrive/0_Practicum2/customdata_yolov8/data.yaml')
print(f"mAP@0.5: {results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {results.box.map:.4f}")


In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load your best-trained model
model_path = '/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_Retrain_April/weights/best.pt'
model = YOLO(model_path)

# Path to test image
image_path = '/content/drive/MyDrive/11_LicensePlateDetection/test_image.jpg'

# Read and convert image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run inference
results = model(image_path, conf=0.25, verbose=False)[0]

# Draw bounding boxes and confidence scores
for box, score in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
    x1, y1, x2, y2 = map(int, box[:4])
    confidence = score * 100
    label = f"{confidence:.1f}%"

    cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)
    cv2.putText(image_rgb, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), 2)

# Show result
plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("YOLOv8 Inference with best.pt")
plt.show()


In [ ]:
# Load your best-trained model
model_path = '/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_Retrain_April/weights/best.pt'
model = YOLO(model_path)

# Path to test image
image_path = '/content/drive/MyDrive/11_LicensePlateDetection/test_image.jpg'

# Read and convert image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run inference
results = model(image_path, conf=0.25, verbose=False)[0]

# Draw bounding boxes and confidence scores
for box, score in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
    x1, y1, x2, y2 = map(int, box[:4])
    confidence = score * 100
    label = f"{confidence:.1f}%"

    # Draw rectangle around the plate
    cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Text background box
    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)
    cv2.rectangle(image_rgb, (x1, y1 - text_height - 10), (x1 + text_width, y1), (0, 255, 0), -1)

    # Put readable confidence text over it
    cv2.putText(image_rgb, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)

# Show result
plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("YOLOv8 Inference with best.pt")
plt.show()


In [ ]:
# Load your best-trained model
model_path = '/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_Retrain_April/weights/best.pt'
model = YOLO(model_path)

# Path to test image
image_path = '/content/drive/MyDrive/0_Practicum2/Nissan_Images/10FT.jpg'

# Read and convert image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run inference
results = model(image_path, conf=0.15, verbose=False)[0]

# Draw bounding boxes and confidence scores
for box, score in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
    x1, y1, x2, y2 = map(int, box[:4])
    confidence = score * 100
    label = f"{confidence:.1f}%"

    # Draw rectangle around the plate
    cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Text background box
    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)
    cv2.rectangle(image_rgb, (x1, y1 - text_height - 10), (x1 + text_width, y1), (0, 255, 0), -1)

    # Put readable confidence text over it
    cv2.putText(image_rgb, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)

# Show result
plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("YOLOv8 Inference with best.pt")
plt.show()

# Detection on Directory Images

In [ ]:
import os
import glob

# Load trained model
model_path = '/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_Retrain_April/weights/best.pt'
model = YOLO(model_path)

# Directory with test images
image_dir = '/content/drive/MyDrive/0_Practicum2/Nissan_Images/'
image_paths = glob.glob(os.path.join(image_dir, '*.jpg'))

# Loop through each image
for image_path in image_paths:
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Run inference
    results = model(image_path, conf=0.15, verbose=False)[0]

    # Draw bounding boxes and large confidence text
    for box, score in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
        x1, y1, x2, y2 = map(int, box[:4])
        confidence = score * 100
        label = f"{confidence:.1f}%"

        # Draw rectangle
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)

        # Background for text
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 2.0, 4)
        cv2.rectangle(image_rgb, (x1, y1 - text_height - 12), (x1 + text_width, y1), (0, 255, 0), -1)

        # Overlay confidence label
        cv2.putText(image_rgb, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 0), 4)

    # Display results
    plt.figure(figsize=(14, 10))
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.title(f"YOLOv8 Inference - {os.path.basename(image_path)}")
    plt.show()


In [ ]:
# Load your best-trained model
model_path = '/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_Retrain_April/weights/best.pt'
model = YOLO(model_path)

# Path to test image
image_path = '/content/drive/MyDrive/11_LicensePlateDetection/test_image2.jpg'

# Read and convert image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run inference
results = model(image_path, conf=0.15, verbose=False)[0]

# Draw bounding boxes and confidence scores
for box, score in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
        x1, y1, x2, y2 = map(int, box[:4])
        confidence = score * 100
        label = f"{confidence:.1f}%"

        # Draw rectangle
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)

        # Background for text
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 2.0, 4)
        cv2.rectangle(image_rgb, (x1, y1 - text_height - 12), (x1 + text_width, y1), (0, 255, 0), -1)

        # Overlay confidence label
        cv2.putText(image_rgb, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 0), 4)

# Show result
plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("YOLOv8 Inference with best.pt")
plt.show()

# New Dataset

In [ ]:
# Define paths
import os
import shutil

project_name = "30_April_Run"
drive_path = "/content/drive/MyDrive/0_Practicum2"
os.makedirs(drive_path, exist_ok=True)

# Download and unzip dataset directly to Drive
#!curl -L "xxx" > roboflow.zip
!unzip -q roboflow.zip -d "{os.path.join(drive_path, project_name)}" # Unzip directly to drive path
!rm roboflow.zip # Remove the zip file

In [ ]:
# Load model
model = YOLO("/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_Retrain_April/weights/last.pt")

# Start a new training run instead of resuming
model.train(
    data='/content/drive/MyDrive/0_Practicum2/30_April_Run/data.yaml',
    epochs=30,
    imgsz=1024,
    batch=8,
    resume=False,
    name='v8_New_data_Model',
    project='/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs'
)

In [ ]:
model.val(data='/content/drive/MyDrive/0_Practicum2/30_April_Run/data.yaml')


In [ ]:
model = YOLO('/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_New_data_Model/weights/best.pt')
results = model.val(data='/content/drive/MyDrive/0_Practicum2/30_April_Run/data.yaml')
print(f"mAP@0.5: {results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {results.box.map:.4f}")

# Evaluate


In [ ]:

model_path = '/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_Retrain_April/weights/best.pt'
model = YOLO(model_path)

# Path to test image
image_path = '/content/drive/MyDrive/11_LicensePlateDetection/test_image2.jpg'

# Read and convert image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run inference
results = model(image_path, conf=0.15, verbose=False)[0]

# Draw bounding boxes and confidence scores
for box, score in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
        x1, y1, x2, y2 = map(int, box[:4])
        confidence = score * 100
        label = f"{confidence:.1f}%"

        # Draw rectangle
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)

        # Background for text
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 2.0, 4)
        cv2.rectangle(image_rgb, (x1, y1 - text_height - 12), (x1 + text_width, y1), (0, 255, 0), -1)

        # Overlay confidence label
        cv2.putText(image_rgb, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 0), 4)

# Show result
plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("YOLOv8 Inference with best.pt")
plt.show()

In [ ]:
import os
import glob

# Load trained model
model_path = '/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_New_data_Model/weights/best.pt'
model = YOLO(model_path)

# Directory with test images
image_dir = '/content/drive/MyDrive/0_Practicum2/Nissan_Images/'
image_paths = glob.glob(os.path.join(image_dir, '*.jpg'))

# Loop through each image
for image_path in image_paths:
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Run inference
    results = model(image_path, conf=0.35, verbose=False)[0]

    # Draw bounding boxes and large confidence text
    for box, score in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
        x1, y1, x2, y2 = map(int, box[:4])
        confidence = score * 100
        label = f"{confidence:.1f}%"

        # Draw rectangle
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)

        # Background for text
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 2.0, 4)
        cv2.rectangle(image_rgb, (x1, y1 - text_height - 12), (x1 + text_width, y1), (0, 255, 0), -1)

        # Overlay confidence label
        cv2.putText(image_rgb, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 0), 4)

    # Display results
    plt.figure(figsize=(14, 10))
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.title(f"YOLOv8 Inference - {os.path.basename(image_path)}")
    plt.show()

In [ ]:

model_path = '/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_New_data_Model/weights/best.pt'
model = YOLO(model_path)

# Path to test image
image_path = '/content/drive/MyDrive/0_Practicum2/Nissan_Images/BasePic.jpg'

# Read and convert image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run inference
results = model(image_path, conf=0.45, verbose=False)[0]

# Draw bounding boxes and confidence scores
for box, score in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
        x1, y1, x2, y2 = map(int, box[:4])
        confidence = score * 100
        label = f"{confidence:.1f}%"

        # Draw rectangle
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)

        # Background for text
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 2.0, 4)
        cv2.rectangle(image_rgb, (x1, y1 - text_height - 12), (x1 + text_width, y1), (0, 255, 0), -1)

        # Overlay confidence label
        cv2.putText(image_rgb, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 0), 4)

# Show result
plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("YOLOv8 Inference with best.pt")
plt.show()

In [ ]:

model_path = '/content/drive/MyDrive/0_Practicum2/customdata_yolov8/runs/v8_New_data_Model/weights/best.pt'
model = YOLO(model_path)

# Path to test image
image_path = '/content/drive/MyDrive/0_Practicum2/Nissan_Images/10FT.jpg'

# Read and convert image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run inference
results = model(image_path, conf=0.45, verbose=False)[0]

# Draw bounding boxes and confidence scores
for box, score in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
    x1, y1, x2, y2 = map(int, box[:4])
    confidence = score * 100
    label = f"{confidence:.1f}%"

    # Draw rectangle around the plate
    cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Text background box
    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)
    cv2.rectangle(image_rgb, (x1, y1 - text_height - 10), (x1 + text_width, y1), (0, 255, 0), -1)

    # Put readable confidence text over it
    cv2.putText(image_rgb, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)

# Show result
plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("YOLOv8 Inference with best.pt")
plt.show()